<a href="https://colab.research.google.com/github/byshadowoz/CCFDM/blob/main/TestModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
pip install --upgrade pip


In [63]:
pip install scikit-learn


In [64]:
import pickle

In [65]:
with open('CCFD.pkl', 'rb') as archivo:
    modeloCCF = pickle.load(archivo)

In [66]:
modeloCCF

LogisticRegression()

In [67]:
import sqlite3
import pandas as pd

In [76]:
conn = sqlite3.connect('/content/bank.db')
query = "SELECT * FROM creditcard"
df = pd.read_sql_query(query, conn, index_col='id')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1 to 1
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   cc_num                 1 non-null      object        
 1   merchant               1 non-null      object        
 2   category               1 non-null      object        
 3   amt                    1 non-null      float64       
 4   city                   1 non-null      object        
 5   state                  1 non-null      object        
 6   job                    1 non-null      object        
 7   trans_date_trans_time  1 non-null      datetime64[ns]
 8   Time                   1 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 80.0+ bytes


In [78]:
df

,cc_num,merchant,category,amt,city,state,job,trans_date_trans_time
id,,,,,,,,
1,3573030000000000,fraud_Kirlin and Sons,travel,201.5,Thomas,WV,Biomedical engineer,21/06/2020 12:15


In [71]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [79]:
import numpy as np
import pandas as pd

def processingDT(df, columns):
  #creating new column with the time btween the time in the column and the first column (first column = 0)
  if 'trans_date_trans_time' in df.columns:
    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
    df['Time'] = (df['trans_date_trans_time'] - df['trans_date_trans_time'].iloc[0]).dt.total_seconds()
    df['Time'].fillna(0, inplace=True)
    df['Time'] = df['Time'].astype(float)
    df['Time'] = df['Time'] / 60

  #drop unuseful data for the ML model
  df = df.drop('trans_date_trans_time', axis=1)
  #Processing string data to boolean data
  def dummies(df,columns):

    dummies = pd.get_dummies(df[columns])

    df = pd.concat([df, dummies], axis=1)

    df.drop(columns=columns, inplace=True)

    return df

  df = dummies(df,[ 'merchant', 'category','city', 'state', 'job',])

  x_cols =list(df.columns)
  x_cols.remove('Time')
  x_cols.remove('amt')

  ss = StandardScaler()
  df[x_cols] = ss.fit_transform(df[x_cols])

  pca2 = PCA(n_components=28, random_state=42)
  pca2 = pca2.fit_transform(df[x_cols])

  columns = ['V' + str(i) for i in range(1, 29)]
  data2 = pd.DataFrame(pca2, columns=columns).assign(Time=df['Time'],Amount=df['amt'])

  return df


In [80]:
processingDT(df, ['cc_num', 'merchant', 'category', 'amt', 'city', 'state', 'job',
       'trans_date_trans_time'])

<ipython-input-79-ce5a6990b2d8>:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])


ValueError: n_components=28 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'